In [ ]:
# Install necessary packages
!pip install googletrans==4.0.0-rc1 gtts ipywidgets -q
print("All libraries installed successfully!")

All libraries installed successfully!


In [ ]:
# Import all required libraries
import ipywidgets as widgets
from IPython.display import display, HTML, Audio, clear_output
from googletrans import Translator, LANGUAGES
import gtts
import io
import base64

print("All libraries imported successfully!")

All libraries imported successfully!


In [ ]:
translator = Translator()

# Create a dictionary of language codes and names
language_dict = {code: name.capitalize() for code, name in LANGUAGES.items()}

# Sort all languages alphabetically by name
sorted_languages = sorted(language_dict.items(), key=lambda x: x[1])

# Define popular languages for quick access (Name, Code) format
popular_langs = [
    ('English', 'en'),
    ('Spanish', 'es'),
    ('French', 'fr'),
    ('German', 'de'),
    ('Italian', 'it'),
    ('Portuguese', 'pt'),
    ('Russian', 'ru'),
    ('Japanese', 'ja'),
    ('Korean', 'ko'),
    ('Chinese (Simplified)', 'zh-cn'),
    ('Arabic', 'ar'),
    ('Hindi', 'hi'),
    ('Urdu', 'ur')
]

# Convert sorted_languages to (Name, Code) format for consistency
sorted_languages_formatted = [(name, code) for code, name in sorted_languages]

# Combine popular and sorted languages for the target dropdown
target_lang_options = popular_langs + sorted_languages_formatted

print("Translator initialized and language data prepared!")
print(f"Total languages available: {len(LANGUAGES)}")

Translator initialized and language data prepared!
Total languages available: 107


In [ ]:
# Create UI Components - Input and Dropdowns
# Header

print("Language Translation Tool")
print("=" * 50)

# Input text area
input_text = widgets.Textarea(
    value='',
    placeholder='Enter text to translate...',
    description='Input:',
    layout=widgets.Layout(width='90%', height='120px'),
    style={'description_width': '80px'}
)

# Source language dropdown (with auto-detect option)
source_lang = widgets.Dropdown(
    options=[('Auto-detect', 'auto')] + popular_langs + sorted_languages_formatted,
    value='auto',
    description='From:',
    layout=widgets.Layout(width='45%'),
    style={'description_width': '80px'}
)

# Target language dropdown
target_lang = widgets.Dropdown(
    options=target_lang_options,
    description='To:',
    layout=widgets.Layout(width='45%'),
    style={'description_width': '80px'}
)

# Set default target language to English
default_target_lang_value = 'en'
found_default_value = None

for label, value in target_lang_options:
    if value == default_target_lang_value:
        found_default_value = value
        break

if found_default_value is not None:
    target_lang.value = found_default_value
else:
    print(f" Warning: Default target language '{default_target_lang_value}' not found.")
    if target_lang_options:
        target_lang.value = target_lang_options[0][1]

print("Input components created!")

Language Translation Tool
Input components created!


In [ ]:
# Create UI Components - Buttons
# Translate button

translate_btn = widgets.Button(
    description='🔄 Translate',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px'),
    style={'font_weight': 'bold'}
)

# Copy button
copy_btn = widgets.Button(
    description='📋 Copy Translation',
    button_style='info',
    layout=widgets.Layout(width='200px', height='40px'),
    disabled=True
)

# Text-to-speech button
tts_btn = widgets.Button(
    description='🔊 Listen',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px'),
    disabled=True
)

# Clear button
clear_btn = widgets.Button(
    description='🗑️ Clear',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)

print("All buttons created!")

All buttons created!


In [ ]:
# Create Output Components and Global Variables
# Output area for translated text

output_area = widgets.HTML(
    value='<div style="padding: 15px; background-color: #f0f0f0; border-radius: 5px; min-height: 100px;"><i>Translation will appear here...</i></div>'
)

# Status label for messages
status_label = widgets.HTML(value='')

# Global variables to store translation data
translated_text_global = ""
target_lang_code = ""

print("Output components and global variables initialized!")

Output components and global variables initialized!


In [ ]:
# Define Translation Function

def translate_text(b):
    """Translate the input text from source to target language"""
    global translated_text_global, target_lang_code

    # Get user inputs
    text = input_text.value.strip()
    src = source_lang.value
    dest = target_lang.value

    # Validate input
    if not text:
        status_label.value = '<p style="color: red;"> Please enter text to translate!</p>'
        return

    try:
        # Show translating status
        status_label.value = '<p style="color: blue;"> Translating...</p>'

        # Perform translation using Google Translate API
        result = translator.translate(text, src=src, dest=dest)
        translated_text_global = result.text
        target_lang_code = dest

        # Get detected source language (useful for auto-detect)
        detected_lang = result.src
        detected_lang_name = language_dict.get(detected_lang, detected_lang).capitalize()

        # Create styled HTML for output
        output_html = f'''
        <div style="padding: 15px; background-color: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;">
            <h4 style="margin-top: 0; color: #2e7d32;"> Translation:</h4>
            <p style="font-size: 16px; line-height: 1.6;">{result.text}</p>
        </div>
        '''

        # Update status message
        if src == 'auto':
            status_label.value = f'<p style="color: green;"> Detected language: <b>{detected_lang_name}</b></p>'
        else:
            status_label.value = '<p style="color: green;"> Translation completed!</p>'

        # Display the translation
        output_area.value = output_html

        # Enable copy and TTS buttons
        copy_btn.disabled = False
        tts_btn.disabled = False

    except Exception as e:
        # Handle errors
        status_label.value = f'<p style="color: red;">❌ Error: {str(e)}</p>'
        output_area.value = '<div style="padding: 15px; background-color: #ffebee; border-radius: 5px; border-left: 4px solid #f44336;"><p style="color: #c62828;">Translation failed. Please try again.</p></div>'

print("Translation function defined!")

Translation function defined!


In [ ]:
# Define Copy Function

def copy_translation(b):
    """Copy translated text to clipboard"""
    global translated_text_global

    if translated_text_global:
        # Use JavaScript to copy text to clipboard
        js_code = f'''
        <script>
        navigator.clipboard.writeText(`{translated_text_global.replace('`', '\\`')}`).then(function() {{
            console.log('Text copied to clipboard');
        }}, function(err) {{
            console.error('Could not copy text: ', err);
        }});
        </script>
        '''
        display(HTML(js_code))
        status_label.value = '<p style="color: green;"> Copied to clipboard!</p>'

print("Copy function defined!")

Copy function defined!


In [ ]:
# Define Text-to-Speech Function

def speak_translation(b):
    """Convert translated text to speech"""
    global translated_text_global, target_lang_code

    if translated_text_global:
        try:
            status_label.value = '<p style="color: blue;"> Generating audio...</p>'

            # Generate speech using Google Text-to-Speech
            tts = gtts.gTTS(text=translated_text_global, lang=target_lang_code, slow=False)

            # Save audio to bytes buffer
            fp = io.BytesIO()
            tts.write_to_fp(fp)
            fp.seek(0)

            # Create HTML audio player with base64 encoded audio
            audio_html = f'''
            <audio controls autoplay>
                <source src="data:audio/mp3;base64,{base64.b64encode(fp.read()).decode()}" type="audio/mp3">
            </audio>
            '''
            display(HTML(audio_html))

            status_label.value = '<p style="color: green;"> Audio generated!</p>'

        except Exception as e:
            status_label.value = f'<p style="color: red;">❌ TTS Error: {str(e)}</p>'

print("Text-to-Speech function defined!")

Text-to-Speech function defined!


In [ ]:
# Define Clear Function

def clear_all(b):
    """Clear all inputs and outputs, reset to defaults"""
    global translated_text_global, target_lang_code

    # Clear input text
    input_text.value = ''

    # Reset output area
    output_area.value = '<div style="padding: 15px; background-color: #f0f0f0; border-radius: 5px; min-height: 100px;"><i>Translation will appear here...</i></div>'

    # Clear status
    status_label.value = ''

    # Reset language selections
    source_lang.value = 'auto'

    # Reset target language to English
    default_target_lang_value = 'en'
    found_default_value = None

    for label, value in target_lang_options:
        if value == default_target_lang_value:
            found_default_value = value
            break

    if found_default_value is not None:
        target_lang.value = found_default_value
    elif target_lang_options:
        target_lang.value = target_lang_options[0][1]

    # Disable copy and TTS buttons
    copy_btn.disabled = True
    tts_btn.disabled = True

    # Clear global variables
    translated_text_global = ""
    target_lang_code = ""

print("Clear function defined!")

Clear function defined!


In [ ]:
# Attach Event Handlers to Buttons
# Connect button click events to their respective functions

translate_btn.on_click(translate_text)
copy_btn.on_click(copy_translation)
tts_btn.on_click(speak_translation)
clear_btn.on_click(clear_all)

print("Event handlers attached to buttons!")

Event handlers attached to buttons!


In [ ]:
# Create Layout and Display UI
# Create horizontal box for language dropdowns

lang_box = widgets.HBox(
    [source_lang, target_lang],
    layout=widgets.Layout(justify_content='space-between')
)

# Create horizontal box for all buttons
button_box = widgets.HBox(
    [translate_btn, copy_btn, tts_btn, clear_btn],
    layout=widgets.Layout(justify_content='space-around')
)

# Combine all components into a vertical layout
ui_layout = widgets.VBox([
    input_text,
    lang_box,
    button_box,
    status_label,
    output_area
], layout=widgets.Layout(padding='20px'))

# Display the complete UI
display(ui_layout)

# Print usage tips
print("\n Tips:")
print("  • Select 'Auto-detect' to automatically identify the source language")
print("  • Use the Copy button to copy translated text to clipboard")
print("  • Use the Listen button to hear the translation")
print("  • Popular languages are listed at the top of dropdowns")


 Tips:
  • Select 'Auto-detect' to automatically identify the source language
  • Use the Copy button to copy translated text to clipboard
  • Use the Listen button to hear the translation
  • Popular languages are listed at the top of dropdowns
